# Spatial ETL Notebook UI
Use this notebook to authenticate, configure, and run the ETL pipeline without editing YAML files.

1. Run the first cell to ensure project paths are set (and optionally trigger authentication).
2. Run the second cell to launch the widget UI, choose AOI/variables/year/season/CRS/storage, and execute the job.

Notes:
- If widgets do not render, ensure `ipywidgets` is installed and enabled in your Jupyter environment.
- For Copernicus openEO TLS issues, see the note in Cell 1 about `OPENEO_VERIFY_SSL`.


In [ ]:
# Cell 1: environment/authentication setup
import sys
import os
import subprocess
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / 'src').exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
SRC_PATH = PROJECT_ROOT / 'src'
if SRC_PATH.exists() and str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

print('Project root:', PROJECT_ROOT)

# If you hit TLS/certificate errors with Copernicus Data Space openEO, you can disable
# verification temporarily (insecure). Uncomment to force:
# os.environ['OPENEO_VERIFY_SSL'] = '0'
if os.environ.get('OPENEO_VERIFY_SSL') in {'0', 'false', 'no'}:
    print('WARNING: OPENEO_VERIFY_SSL=0 -> TLS verification disabled (insecure).')

# Performance: run up to 2 tasks in parallel (openEO free tier limit).
os.environ.setdefault('SDM_MAX_CONCURRENT_TASKS', '2')
print('Max concurrent tasks:', os.environ.get('SDM_MAX_CONCURRENT_TASKS'))

print('Auth notes:')
print('- Sentinel-2 indices (ndvi/ndmi/msi/bsi) use Copernicus Data Space openEO.')
print('  If no openEO session is found, you will be prompted to authenticate (OIDC device flow).')
print('- AlphaEarth uses Google Earth Engine: run earthengine authenticate once if you use alpha_earth.')
print('- GCS uploads use Application Default Credentials: run gcloud auth application-default login.')

RUN_GEE_AUTH = False
if RUN_GEE_AUTH:
    print('Starting Earth Engine auth flow...')
    subprocess.run(['earthengine', 'authenticate'], check=True)
else:
    print('Set RUN_GEE_AUTH=True above to run Earth Engine auth here, or run earthengine authenticate in a terminal.')

GCLOUD_EXE = r"C:\Users\usuario\AppData\Local\Google\Cloud SDK\google-cloud-sdk\bin\gcloud.cmd"

RUN_GCS_AUTH = True
if RUN_GCS_AUTH:
    if not os.path.exists(GCLOUD_EXE):
        print(f"gcloud not found at {GCLOUD_EXE}. Update GCLOUD_EXE or install Google Cloud SDK.")
    else:
        print('Starting gcloud Application Default Credentials auth...')
        subprocess.run([GCLOUD_EXE, 'auth', 'application-default', 'login'], check=True)
else:
    print('Set RUN_GCS_AUTH=True above to run gcloud ADC auth here, or run gcloud auth application-default login in a terminal.')


Project root: C:\Users\usuario\Documents\GitHub\spatial_data_mining
Max concurrent tasks: 2
Auth notes:
- Sentinel-2 indices (ndvi/ndmi/msi/bsi) use Copernicus Data Space openEO.
  If no openEO session is found, you will be prompted to authenticate (OIDC device flow).
- AlphaEarth uses Google Earth Engine: run earthengine authenticate once if you use alpha_earth.
Set RUN_GEE_AUTH=True above to run Earth Engine auth here, or run earthengine authenticate in a terminal.


In [ ]:
# Cell 2: interactive UI to configure and run the pipeline
import sys
import yaml
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path
import importlib
from datetime import datetime
import threading
import time

try:
    from ipyfilechooser import FileChooser
    HAS_FILE_CHOOSER = True
except Exception:
    FileChooser = None
    HAS_FILE_CHOOSER = False

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / 'src').exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
SRC_PATH = PROJECT_ROOT / 'src'
if SRC_PATH.exists() and str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

import spatial_data_mining.config as cfg
import spatial_data_mining.extract.openeo_indices as openeo_indices
import spatial_data_mining.extract.openeo_rgb as openeo_rgb
import spatial_data_mining.extract.openeo_swi as openeo_swi
import spatial_data_mining.transform.raster_ops as raster_ops
import spatial_data_mining.variables.registry as registry
import spatial_data_mining.orchestrator as orchestrator
importlib.reload(cfg)
importlib.reload(openeo_indices)
importlib.reload(openeo_rgb)
importlib.reload(openeo_swi)
importlib.reload(raster_ops)
importlib.reload(registry)
importlib.reload(orchestrator)
from spatial_data_mining.orchestrator import run_pipeline_from_dict
from spatial_data_mining.utils.cancellation import PipelineCancelled
import spatial_data_mining.variables.metadata as metadata
importlib.reload(metadata)
from spatial_data_mining.variables.metadata import VARIABLE_METADATA, get_variable_metadata

BASE_CONFIG_PATH = PROJECT_ROOT / 'config' / 'base.yaml'
AOI_DIR = PROJECT_ROOT / 'data' / 'aoi'
DEFAULT_OUTPUT_DIR = PROJECT_ROOT / 'data' / 'outputs'

VARIABLE_OPTIONS = list(VARIABLE_METADATA.keys())
DEFAULT_SEASONS = ['winter', 'spring', 'summer', 'autumn', 'annual']
DEFAULT_SEASON_LABEL = 'Seasons'
CLCPLUS_NAME = 'clcplus'
SWI_NAME = 'swi'
RGB_NAMES = {'rgb', 'rgb_raw'}


def load_defaults():
    if BASE_CONFIG_PATH.exists():
        with BASE_CONFIG_PATH.open('r', encoding='utf-8') as f:
            data = yaml.safe_load(f) or {}
    else:
        data = {}
    defaults = data.get('defaults', {})
    allowed_crs = defaults.get('allowed_crs', ['EPSG:4326'])
    resolution = defaults.get('resolution_m', 20)
    storage = defaults.get('storage', {'kind': 'local_cog', 'output_dir': str(DEFAULT_OUTPUT_DIR)})
    out_dir = storage.get('output_dir') or DEFAULT_OUTPUT_DIR
    out_dir = Path(out_dir)
    if not out_dir.is_absolute():
        out_dir = PROJECT_ROOT / out_dir
    storage['output_dir'] = str(out_dir)
    return allowed_crs, resolution, storage


def list_aois():
    if not AOI_DIR.exists():
        return []
    return sorted(p for p in AOI_DIR.glob('*') if p.is_file())


def year_range_for_variable(var_name: str):
    meta = get_variable_metadata(var_name)
    cov = meta.get('temporal_coverage', {})
    start = cov.get('start_year', 2000)
    end = cov.get('end_year', 'present')
    if end == 'present':
        end = datetime.now().year
    return start, int(end)


def season_options_for_variable(var_name: str):
    meta = get_variable_metadata(var_name)
    seasons = meta.get('season_options')
    return seasons if seasons else DEFAULT_SEASONS


def default_swi_date_for(season: str, year: int):
    if str(season).lower() == 'static':
        return None
    try:
        start_str, end_str = openeo_indices.season_date_range(int(year), season)
        start_dt = datetime.fromisoformat(start_str).date()
        end_dt = datetime.fromisoformat(end_str).date()
        return start_dt + (end_dt - start_dt) // 2
    except Exception:
        return None


def intersect_years(vars_selected):
    if not vars_selected:
        return list(range(2000, datetime.now().year + 1))
    starts = []
    ends = []
    for v in vars_selected:
        s, e = year_range_for_variable(v)
        starts.append(s)
        ends.append(e)
    start_max = max(starts)
    end_min = min(ends)
    if start_max > end_min:
        return []
    return list(range(start_max, end_min + 1))


def intersect_seasons(vars_selected):
    if not vars_selected:
        return DEFAULT_SEASONS
    season_sets = [set(season_options_for_variable(v)) for v in vars_selected]
    common = set.intersection(*season_sets) if season_sets else set(DEFAULT_SEASONS)
    if common:
        ordered_common = [s for s in DEFAULT_SEASONS if s in common]
        extras = [s for s in common if s not in ordered_common]
        return ordered_common + extras
    return ['annual']


def clcplus_selected(vars_selected=None):
    values = vars_selected if vars_selected is not None else variables.value
    return any(str(v).lower() == CLCPLUS_NAME for v in values)


def swi_selected(vars_selected=None):
    values = vars_selected if vars_selected is not None else variables.value
    return any(str(v).lower() == SWI_NAME for v in values)


def rgb_selected(vars_selected=None):
    values = vars_selected if vars_selected is not None else variables.value
    return any(str(v).lower() in RGB_NAMES for v in values)


def date_required(vars_selected=None):
    return swi_selected(vars_selected) or rgb_selected(vars_selected)


allowed_crs, default_resolution, storage_defaults = load_defaults()
aoi_paths = list_aois()
aoi_options = [p.name for p in aoi_paths] if aoi_paths else []
aoi_map = {p.name: p for p in aoi_paths}

job_name = widgets.Text(value='notebook_job', description='Job name')
aoi_select = widgets.SelectMultiple(options=aoi_options, description='AOIs', rows=6)
target_crs = widgets.Dropdown(options=allowed_crs, description='Target CRS')
use_native_res = widgets.Checkbox(value=False, description='Use native resolution')
resolution = widgets.FloatText(value=default_resolution, description='Resolution (m)')
initial_years = list(range(2000, datetime.now().year + 1))
years = widgets.SelectMultiple(options=initial_years, description='Years', rows=6)
seasons = widgets.SelectMultiple(options=DEFAULT_SEASONS, value=('summer',), description=DEFAULT_SEASON_LABEL, rows=5)
variables = widgets.SelectMultiple(
    options=VARIABLE_OPTIONS,
    value=('ndvi',),
    description='Variables'
)
storage_kind = widgets.ToggleButtons(
    options=[('Local COG', 'local_cog'), ('GCS COG', 'gcs_cog')],
    value='local_cog',
    description='Storage'
)
output_dir = widgets.Text(value=storage_defaults.get('output_dir', str(DEFAULT_OUTPUT_DIR)), description='Output dir')
choose_output_btn = widgets.Button(description='Browse output dir', icon='folder-open')
choose_output_btn.layout.width = '200px'
choose_output_btn.style.button_color = '#e8e8e8'
gcs_bucket = widgets.Text(value=storage_defaults.get('bucket', 'your-bucket'), description='GCS bucket')
gcs_prefix = widgets.Text(value=storage_defaults.get('prefix', 'spatial/outputs'), description='GCS prefix')
run_button = widgets.Button(description='Run pipeline', button_style='primary')
status = widgets.HTML(value='<b>Status:</b> idle')
pause_button = widgets.ToggleButton(value=False, description='Pause', button_style='warning')
pause_button.layout.width = '120px'
pause_button.disabled = True
stop_button = widgets.Button(description='Stop', button_style='danger')
stop_button.layout.width = '120px'
stop_button.disabled = True

pipeline_thread = None
stop_event = None
pause_event = None
log_output = widgets.Output()
clcplus_dir = widgets.Text(value='', description='CLCplus dir')
choose_clcplus_btn = widgets.Button(description='Browse CLCplus dir', icon='folder-open')
choose_clcplus_btn.layout.width = '220px'
choose_clcplus_btn.style.button_color = '#e8e8e8'

swi_date = widgets.DatePicker(description='Date (within season)')

chooser_box = widgets.VBox()
dir_chooser = None
if HAS_FILE_CHOOSER:
    dir_chooser = FileChooser(str(Path(output_dir.value).expanduser()), select_default=True, show_only_dirs=True)
    dir_chooser.title = 'Select output directory'
    dir_chooser.use_dir_icons = True
    def on_dir_select(chooser):
        if chooser.selected_path:
            output_dir.value = chooser.selected_path
    dir_chooser.register_callback(on_dir_select)
    chooser_box.children = (dir_chooser,)
    chooser_box.layout.display = 'none'

clcplus_chooser_box = widgets.VBox()
clcplus_dir_chooser = None
if HAS_FILE_CHOOSER:
    clcplus_dir_chooser = FileChooser(str(PROJECT_ROOT), select_default=True, show_only_dirs=True)
    clcplus_dir_chooser.title = 'Select CLCplus folder'
    clcplus_dir_chooser.use_dir_icons = True
    def on_clcplus_select(chooser):
        if chooser.selected_path:
            clcplus_dir.value = chooser.selected_path
    clcplus_dir_chooser.register_callback(on_clcplus_select)
    clcplus_chooser_box.children = (clcplus_dir_chooser,)
    clcplus_chooser_box.layout.display = 'none'

storage_box = widgets.VBox()
output_picker_box = widgets.VBox()
clcplus_box = widgets.VBox()
swi_box = widgets.VBox()


def refresh_storage_fields(change=None):
    output_children = [widgets.HBox([output_dir, choose_output_btn])]
    if HAS_FILE_CHOOSER and dir_chooser:
        output_children.append(chooser_box)
    output_picker_box.children = tuple(output_children)
    if storage_kind.value == 'local_cog':
        storage_box.children = (output_picker_box,)
    else:
        storage_box.children = (output_picker_box, widgets.HBox([gcs_bucket, gcs_prefix]))


def on_native_change(change):
    resolution.disabled = change['new']


def choose_output_dir(_):
    if HAS_FILE_CHOOSER and dir_chooser:
        chooser_box.layout.display = None
        dir_chooser.reset(path=str(Path(output_dir.value).expanduser()))
        return
    try:
        import tkinter as tk
        from tkinter import filedialog
        root = tk.Tk()
        root.withdraw()
        chosen = filedialog.askdirectory(initialdir=output_dir.value or str(PROJECT_ROOT))
        root.destroy()
        if chosen:
            output_dir.value = chosen
    except Exception as exc:
        with log_output:
            print('Directory picker unavailable in this environment. Please type the path manually.', exc)


def choose_clcplus_dir(_):
    if HAS_FILE_CHOOSER and clcplus_dir_chooser:
        clcplus_chooser_box.layout.display = None
        clcplus_dir_chooser.reset(path=str(Path(clcplus_dir.value or PROJECT_ROOT).expanduser()))
        return
    try:
        import tkinter as tk
        from tkinter import filedialog
        root = tk.Tk()
        root.withdraw()
        chosen = filedialog.askdirectory(initialdir=clcplus_dir.value or str(PROJECT_ROOT))
        root.destroy()
        if chosen:
            clcplus_dir.value = chosen
    except Exception as exc:
        with log_output:
            print('CLCplus directory picker unavailable. Please type the path manually.', exc)


def refresh_clcplus_fields(_=None):
    clcplus_box.layout.display = None if clcplus_selected() else 'none'


def refresh_swi_fields(_=None):
    active = date_required()
    swi_box.layout.display = None if active else 'none'
    seasons.description = 'Season (date)' if active else DEFAULT_SEASON_LABEL
    if active:
        selected_years = tuple(years.value) if getattr(years, 'value', None) else ()
        selected_seasons = tuple(seasons.value) if getattr(seasons, 'value', None) else ()
        single = len(selected_years) == 1 and len(selected_seasons) == 1
        swi_date.disabled = not single
        swi_date.description = 'Date (within season)' if single else 'Date (auto per season)'
    else:
        swi_date.disabled = False
        swi_date.description = 'Date (within season)'


def update_swi_date_default(_=None):
    if not date_required():
        return
    selected_years = tuple(years.value) if getattr(years, 'value', None) else ()
    selected_seasons = tuple(seasons.value) if getattr(seasons, 'value', None) else ()
    if len(selected_years) != 1 or len(selected_seasons) != 1:
        return
    year = int(selected_years[0])
    season = selected_seasons[0]
    default_date = default_swi_date_for(season, year)
    if default_date is None:
        return
    try:
        start_str, end_str = openeo_indices.season_date_range(year, season)
        start_dt = datetime.fromisoformat(start_str).date()
        end_dt = datetime.fromisoformat(end_str).date()
    except Exception:
        start_dt = end_dt = None
    current = swi_date.value
    if current is None or (start_dt and end_dt and not (start_dt <= current <= end_dt)):
        swi_date.value = default_date


def update_time_controls(_=None):
    vars_selected = list(variables.value)
    years_opts = intersect_years(vars_selected)
    if not years_opts:
        years_opts = list(range(datetime.now().year, datetime.now().year + 1))
    years_sorted = sorted(years_opts)
    prev_years = set(years.value) if getattr(years, 'value', ()) else set()
    selected_years = [y for y in years_sorted if y in prev_years] or [years_sorted[-1]]
    with years.hold_trait_notifications():
        years.value = ()
        years.options = years_sorted
        years.value = tuple(selected_years)

    seasons_opts = intersect_seasons(vars_selected)
    prev_seasons = set(seasons.value) if getattr(seasons, 'value', ()) else set()
    selected_seasons = [s for s in seasons_opts if s in prev_seasons] or seasons_opts[:1]
    with seasons.hold_trait_notifications():
        seasons.value = ()
        seasons.options = seasons_opts
        seasons.value = tuple(selected_seasons)

    refresh_clcplus_fields()
    refresh_swi_fields()
    update_swi_date_default()


def build_clcplus_box():
    if HAS_FILE_CHOOSER and clcplus_dir_chooser:
        clcplus_box.children = (widgets.VBox([widgets.HBox([clcplus_dir, choose_clcplus_btn]), clcplus_chooser_box]),)
    else:
        clcplus_box.children = (widgets.HBox([clcplus_dir, choose_clcplus_btn]),)


def build_swi_box():
    swi_box.children = (swi_date,)


def get_aoi_paths():
    selected = list(aoi_select.value)
    return [str(aoi_map[name].resolve()) for name in selected if name in aoi_map]


def log(message: str):
    try:
        log_output.append_stdout(str(message) + '\n')
    except Exception:
        print(message)


def set_status(state: str):
    status.value = f"<b>Status:</b> {state}"


def progress(message: str):
    log(message)


def _set_controls_running(running: bool, widgets_to_disable=None):
    if widgets_to_disable:
        for w in widgets_to_disable:
            try:
                w.disabled = running
            except Exception:
                pass
    try:
        run_button.disabled = running
    except Exception:
        pass
    for w in (pause_button, stop_button):
        try:
            w.disabled = not running
        except Exception:
            pass


def on_pause_toggled(change):
    global pause_event, stop_event, pipeline_thread
    if pause_button.disabled:
        return
    if stop_event is not None and stop_event.is_set():
        return
    if not pipeline_thread or not pipeline_thread.is_alive() or pause_event is None:
        pause_button.value = False
        pause_button.description = 'Pause'
        pause_button.disabled = True
        return
    if change.get('new'):
        pause_event.set()
        pause_button.description = 'Resume'
        set_status('paused')
        log('Pause requested.')
    else:
        pause_event.clear()
        pause_button.description = 'Pause'
        set_status('running')
        log('Resume requested.')


def on_stop_clicked(_):
    global stop_event, pause_event, pipeline_thread
    if not pipeline_thread or not pipeline_thread.is_alive() or stop_event is None:
        return
    stop_event.set()
    if pause_event is not None:
        pause_event.clear()
    try:
        pause_button.value = False
        pause_button.description = 'Pause'
    except Exception:
        pass
    set_status('stopping')
    log('Stop requested...')


def on_run_clicked(_):
    global pipeline_thread, stop_event, pause_event
    if pipeline_thread is not None and pipeline_thread.is_alive():
        log('Pipeline is already running.')
        return
    log_output.clear_output()
    selected_aoi_paths = get_aoi_paths()
    if not selected_aoi_paths:
        log('Select at least one AOI file from data/aoi/.')
        return
    selected_vars = list(variables.value)
    if not selected_vars:
        log('Select at least one variable before running.')
        return
    selected_years = tuple(years.value) if getattr(years, 'value', None) else ()
    if not selected_years:
        log('Select at least one year.')
        return
    selected_seasons = tuple(seasons.value) if getattr(seasons, 'value', None) else ()
    if not selected_seasons:
        log('Select at least one season.')
        return

    wants_clcplus = clcplus_selected(selected_vars)
    clc_dir_path = None
    if wants_clcplus:
        clc_dir_value = clcplus_dir.value.strip()
        if not clc_dir_value:
            log('Select the CLCplus input folder before running the pipeline.')
            return
        clc_dir_path = Path(clc_dir_value).expanduser()
        if not clc_dir_path.exists() or not clc_dir_path.is_dir():
            log(f'CLCplus folder not found or not a directory: {clc_dir_path}')
            return

    wants_swi = any(str(v).lower() == SWI_NAME for v in selected_vars)
    wants_rgb = any(str(v).lower() in RGB_NAMES for v in selected_vars)
    wants_date = wants_swi or wants_rgb
    date_value = None
    if wants_date:
        single_selection = len(selected_years) == 1 and len(selected_seasons) == 1
        static_selected = any(str(s).lower() == 'static' for s in selected_seasons)
        if static_selected and not (single_selection and swi_date.value):
            log('RGB/SWI with season=static requires an exact date. Select one year and one season, then pick a date.')
            return
        if single_selection and swi_date.value:
            date_value = swi_date.value
            season_value = selected_seasons[0]
            if str(season_value).lower() != 'static':
                start_str, end_str = openeo_indices.season_date_range(int(selected_years[0]), season_value)
                try:
                    start_dt = datetime.fromisoformat(start_str).date()
                    end_dt = datetime.fromisoformat(end_str).date()
                except Exception:
                    start_dt = end_dt = None
                if start_dt and end_dt:
                    if not (start_dt <= date_value <= end_dt):
                        log(f'Date {date_value.isoformat()} is outside the {season_value} {selected_years[0]} range ({start_str}..{end_str}).')
                        return
        else:
            log('RGB/SWI: using default mid-season dates for each year/season.')

    storage_cfg = {'kind': storage_kind.value}
    storage_cfg['output_dir'] = output_dir.value
    if storage_kind.value != 'local_cog':
        storage_cfg['bucket'] = gcs_bucket.value
        storage_cfg['prefix'] = gcs_prefix.value
    resolution_value = None if use_native_res.value else float(resolution.value)
    job_section = {
        'name': job_name.value,
        'aoi_path': selected_aoi_paths[0],
        'aoi_paths': selected_aoi_paths,
        'target_crs': target_crs.value,
        'resolution_m': resolution_value,
        'year': int(selected_years[0]),
        'years': [int(y) for y in selected_years],
        'season': selected_seasons[0],
        'seasons': list(selected_seasons),
        'variables': selected_vars,
        'storage': storage_cfg,
    }
    if wants_clcplus and clc_dir_path:
        job_section['clcplus_input_dir'] = str(clc_dir_path.resolve())
    if wants_swi and date_value:
        job_section['swi_date'] = date_value.isoformat()
    if wants_rgb and date_value:
        job_section['rgb_date'] = date_value.isoformat()

    log(f'Running pipeline for {len(selected_aoi_paths)} AOI(s), {len(selected_vars)} variable(s), {len(selected_years)} year(s), {len(selected_seasons)} season(s)...')

    widgets_to_disable = [
        job_name,
        aoi_select,
        target_crs,
        use_native_res,
        resolution,
        years,
        seasons,
        variables,
        storage_kind,
        output_dir,
        choose_output_btn,
        gcs_bucket,
        gcs_prefix,
        clcplus_dir,
        choose_clcplus_btn,
        swi_date,
    ]

    stop_event = threading.Event()
    pause_event = threading.Event()
    try:
        pause_button.value = False
        pause_button.description = 'Pause'
    except Exception:
        pass
    set_status('running')
    _set_controls_running(True, widgets_to_disable)

    paused_notice = {'sent': False}

    def should_stop():
        if stop_event.is_set():
            return True
        if pause_event.is_set():
            if not paused_notice['sent']:
                log('Pipeline paused.')
                paused_notice['sent'] = True
            while pause_event.is_set() and not stop_event.is_set():
                time.sleep(0.2)
            if paused_notice['sent'] and not stop_event.is_set():
                log('Pipeline resumed.')
            paused_notice['sent'] = False
        return stop_event.is_set()

    def runner():
        try:
            results = run_pipeline_from_dict(job_section, progress_cb=progress, should_stop=should_stop)
        except PipelineCancelled:
            log('Pipeline stopped by user.')
            set_status('stopped')
            return
        except Exception as exc:
            log(f'Pipeline failed: {exc}')
            set_status('failed')
            return
        finally:
            _set_controls_running(False, widgets_to_disable)
            try:
                pause_button.value = False
                pause_button.description = 'Pause'
            except Exception:
                pass

        if not results:
            log('Pipeline completed but returned no outputs (check logs for errors).')
        else:
            log('Pipeline completed. Outputs:')
            for res in results:
                log(f"- {res.get('aoi')} {res.get('variable')} ({res.get('year')} {res.get('season')}): local={res.get('local_path')} gcs={res.get('gcs_uri')}")
        set_status('completed')

    pipeline_thread = threading.Thread(target=runner, daemon=True)
    pipeline_thread.start()

# ensure only one handler is attached
run_button._click_handlers.callbacks = []
run_button.on_click(on_run_clicked)
pause_button.observe(on_pause_toggled, names='value')
stop_button._click_handlers.callbacks = []
stop_button.on_click(on_stop_clicked)
variables.observe(update_time_controls, names='value')
variables.observe(refresh_clcplus_fields, names='value')
years.observe(update_swi_date_default, names='value')
seasons.observe(update_swi_date_default, names='value')
use_native_res.observe(on_native_change, names='value')
storage_kind.observe(refresh_storage_fields, names='value')
choose_output_btn.on_click(choose_output_dir)
choose_clcplus_btn.on_click(choose_clcplus_dir)

# initial setup
on_native_change({'new': use_native_res.value})
refresh_storage_fields()
build_clcplus_box()
build_swi_box()
update_time_controls()
refresh_clcplus_fields()
refresh_swi_fields()
update_swi_date_default()

ui = widgets.VBox([
    job_name,
    aoi_select,
    widgets.HBox([target_crs, resolution, use_native_res]),
    widgets.HBox([years, seasons, swi_box]),
    variables,
    clcplus_box,
    storage_kind,
    storage_box,
    widgets.HBox([run_button, pause_button, stop_button, status]),
    log_output,
])

display(ui)

